In [14]:
import json
from collections import defaultdict

In [15]:
def rename_metric(metric):
    if metric == 'F-measure':
        return 'f1'
    if metric == 'Accuracy':
        return 'acc'
    if metric == 'Runtime':
        return 'time'
    raise ValueError('Unknown metric')

In [16]:
def maybe_add_zero(mean, std):
    if len(mean.split('.')[-1]) == 1:
        mean += '0'
    if len(std.split('.')[-1]) == 1:
        std += '0'
    return mean, std

In [17]:
def get_results(metric, emb_model="TransE"):
    Res = {metric: {'Carcinogenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                   'Mutagenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                    'Semantic Bible': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                    'Vicodi': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''}
                   }}
    for kb in ['carcinogenesis', 'mutagenesis', 'semantic_bible', 'vicodi']:
        for model in ['celoe', 'eltl', 'ecii', 'evolearner', 'NCES', 'NCES_Ensemble']:
            if model not in ['NCES', 'NCES_Ensemble']:
                rename_model = model.upper() if model != 'evolearner' else 'EvoLearner'
                with open(f'datasets/{kb}/Results/{rename_model}_avg_results.json') as file:
                    results = json.load(file)
                mean = str(round(results[metric]['mean'],2)); std = str(round(results[metric]['std'],2))
                mean, std = maybe_add_zero(mean, std)
                Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
            elif model == 'NCES':
                path = f'datasets/{kb}/Results/{model}.json'
                if emb_model == "TransE":
                    path = f'datasets/{kb}/Results/{model}_{emb_model}.json'
                with open(path) as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results: 
                    rename_model = 'NCES$_'+'{\\text{'+m+'}}$' if m != 'SetTransformer' else 'NCES$_'+'{\\text{'+'ST'+'}}$'
                    mean = str(round(results[m][new_metric]['mean'][0],2))
                    std = str(round(results[m][new_metric]['std'][0],2))
                    mean, std = maybe_add_zero(mean, std)
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
            elif model == 'NCES_Ensemble':
                path = f'datasets/{kb}/Results/{model}.json'
                if emb_model == "TransE":
                    path = f'datasets/{kb}/Results/NCES_TransE_Ensemble.json'
                with open(path) as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results:
                    rename_model = 'NCES$_'+'{\\text{'+m+'}}$' if not 'SetTransformer' in m \
                    else 'NCES$_'+'{\\text{'+m.replace('SetTransformer', 'ST')+'}}$'
                    mean = str(round(results[m][new_metric]['mean'][0],2)); std = str(round(results[m][new_metric]['std'][0],2))
                    mean, std = maybe_add_zero(mean, std)
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
    return Res

In [18]:
def write_results(metric, emb_model="TransE"):
    data = get_results(metric, emb_model)
    with open(f'{emb_model}_{metric}.txt', 'w') as file:
        for key in data:
            file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
            file.write("\t\t\\toprule\n")
            file.write("\t\t& \\multicolumn{4}{c}{"+"$"+metric+"(\%)$}\\\\\n")
            file.write("\t\t&"+" & ".join(["\\textbf{"+kb+"}" for kb in data[key]])+"\\\\\n")
            file.write("\\midrule\n")
            for model in data[key]['Carcinogenesis']:
                file.write("\t\t"+model+" & "+" & ".join([data[key][kb][model] for kb in data[key]])+"\\\\\n")

In [19]:
write_results('F-measure')

In [20]:
write_results('Accuracy')

In [21]:
write_results('Runtime')